Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [246]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

#      Lab 2 - TSP

In [247]:
CITIES = pd.read_csv('cities/vanuatu.csv', header=None, names=['name', 'lat', 'lon'],)
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

None

In [248]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

##  First Greedy Algorithm

In [249]:
visited = np.full(len(CITIES), False)
dist = DIST_MATRIX.copy()
city = 0
visited[city] = True
tsp = list()
tsp.append(int(city))

while not np.all(visited):
    dist[:, city] = np.inf
    closest = np.argmin(dist[city])
    visited[closest] = True
    city = closest
    tsp.append(int(city))
tsp.append(tsp[0])
tsp_cost(tsp)

np.float64(1475.528091104531)

##     Second Greedy Algorithm

In [250]:
def cyclic(edges):
    G = nx.Graph()
    G.add_edges_from(edges)
    try:
        nx.find_cycle(G)
        return True
    except:
        return False

In [251]:
segments = [
    ({c1, c2}, float(DIST_MATRIX[c1, c2])) for c1, c2 in combinations(range(len(CITIES)), 2)
]

visited = set()
edges = set()

segments.sort(key=lambda e: e[1])

tsp = []

start_segment = segments.pop(0)
(c1, c2) = tuple(start_segment[0])
visited |= start_segment[0]
edges.add(tuple(start_segment[0]))
tsp.extend([c1, c2])

while len(visited) < len(CITIES):
    for segment in segments:
        cities, distance = segment
        (c1, c2) = tuple(cities)

        if (c1 in visited) ^ (c2 in visited):
            if not cyclic(edges | {tuple(cities)}):
                edges.add((c1, c2))
                visited.add(c1)
                visited.add(c2)
                tsp.append(c2 if c1 in tsp else c1)
                segments.remove(segment)
                break

tsp.append(tsp[0])
tsp_cost(tsp)

np.float64(1481.4725995315478)